In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms


LOG_PATH = "./data/profile_info/"
DATA_DIR = '../data/MNIST'

In [3]:
SCALED_SHAPE = (161, 161)

In [4]:
def load_data(data_dir, batch_size, transform):
    trainset = torchvision.datasets.MNIST(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.MNIST(
        root=data_dir, train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(trainset, batch_size)
    test_loader = torch.utils.data.DataLoader(testset, batch_size)

    return train_loader, test_loader

In [5]:
in_channels = 3
device = "cuda:0"

train_loss = nn.MultiLabelSoftMarginLoss()

learning_rate = 1e-4

batch_size = 256
epoch_num = 5

In [6]:
default_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

scaled_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(SCALED_SHAPE),
    transforms.Normalize((0.5,), (0.5,)),
])

default_train_loader, default_test_loader = load_data(data_dir=DATA_DIR, batch_size=batch_size, transform=default_transform)
scaled_train_loader, scaled_test_loader = load_data(data_dir=DATA_DIR, batch_size=batch_size, transform=scaled_transform)

In [7]:
def train(net, optimizer, criterion, epoch_num, default_train_data, scaled_train_data, device):
    """Neural network training process.
    
    """
    for epoch in range(epoch_num):  # loop over the dataset multiple times
        for ((default_data, _), (scaled_data, _)) in zip(default_train_data, scaled_train_data):
            inputs = default_data.to(device)
            masks = scaled_data.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()    
    return net

In [18]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        self.up1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.up2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU()
        )
        self.up3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(1)
        )



    def forward(self, tensor):
        tensor = self.conv1(tensor) 
        tensor = self.up1(tensor)
        tensor = self.up2(tensor)
        tensor = self.up3(tensor)
        return tensor

In [19]:
net = Network()
net.to(device)

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [20]:
net._modules

OrderedDict([('conv1',
              Sequential(
                (0): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU()
              )),
             ('up1',
              Sequential(
                (0): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
                (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU()
              )),
             ('up2',
              Sequential(
                (0): ConvTranspose2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
                (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU()
              )),
             ('up3',
              Sequential(
                (0): ConvTranspose2d(8, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
               

In [21]:
%%time
net = train(
    net, optimizer, train_loss, epoch_num, default_train_loader, scaled_train_loader, device
)
# default_train_loader, default_test_loader
# scaled_train_loader, scaled_test_loader

RuntimeError: The size of tensor a (161) must match the size of tensor b (217) at non-singleton dimension 3

In [11]:
%%time
for default_data, _ in default_test_loader:
    inputs = default_data.to(device)
    _ = net(inputs)

CPU times: user 1.11 s, sys: 8.01 ms, total: 1.12 s
Wall time: 1.12 s
